In [1]:
!pip install dart-fss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.7/145.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.4.0
    Uninstalling termcolor-2.4.0:
      Successfully uninstalled termcolor-2.4.0


In [3]:
import dart_fss as dart_fss
import pandas as pd
import requests as rq

api_key = '7fb2044ce8063944febcbc64d3b6e2c208d830da'
dart_fss.set_api_key(api_key=api_key)

corp_list = dart_fss.get_corp_list()

corp_list.corps
all = dart_fss.api.filings.get_corp_code()
all[0]

df = pd.DataFrame(all)

df_listed = df[df['stock_code'].notnull()]
df_non_listed = df[df['stock_code'].isnull()]

In [ ]:
### 년도 변경 부탁드립니다.

In [6]:
from tqdm import tqdm

new_df = pd.DataFrame(data=None)

for i in tqdm(range(0,3736)):
    code=df_listed['corp_code'].iloc[i]
    url=f'https://opendart.fss.or.kr/api/exctvSttus.json?crtfc_key=7fb2044ce8063944febcbc64d3b6e2c208d830da&corp_code={code}&bsns_year=2019&reprt_code=11011'
    try:
        data = rq.get(url)
        df=data.json().get('list')
        df=pd.DataFrame(df)
        rgist_exctv_at = df['rgist_exctv_at']
        sex_count = df['sexdstn'].value_counts().get('여', 0)
        corp_name = df['corp_name'].iloc[0]
        new_data = {
            'sex_count': [sex_count],
            'corp_name': [corp_name],
            'rgist_exctv_at' : [rgist_exctv_at]
        }
        new_df_for = pd.DataFrame(new_data)
        new_df = pd.concat([new_df, new_df_for], axis=0)
    except:
        pass

100%|██████████| 3736/3736 [1:08:55<00:00,  1.11s/it]


In [7]:
new_df.to_excel('2019_임원 데이터.xlsx')

In [21]:
from tqdm import tqdm

# 결과 데이터프레임 초기화
result_df = pd.DataFrame(columns=['corp_code', 'corp_name', 'sex', 'position'])

for i in tqdm(range(0,3736)):
    code=df_listed['corp_code'].iloc[i]
    url=f'https://opendart.fss.or.kr/api/exctvSttus.json?crtfc_key=7fb2044ce8063944febcbc64d3b6e2c208d830da&corp_code={code}&bsns_year=2019&reprt_code=11011'
    try:
        # DART API 호출
        response = requests.get(url)
        data = response.json()

        # 임원 정보 추출
        if 'list' in data:
            for exec_info in data['list']:
                corp_name = exec_info['corp_name']
                sex = exec_info['sexdstn']
                position = exec_info['ofcpsNm']
                result_df = result_df.append({'corp_code': corp_code, 'corp_name': corp_name, 'sex': sex, 'position': position}, ignore_index=True)
    except:
        pass

# 결과 데이터프레임 저장
result_df.to_excel('2019임원.xlsx', index=False)

100%|██████████| 3736/3736 [00:00<00:00, 88576.66it/s]


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

# DART API 인증 키 입력
api_key = '7fb2044ce8063944febcbc64d3b6e2c208d830da'

# 회사 코드 목록 가져오기
df_listed = pd.read_csv('listed_companies.csv')

# 결과 데이터프레임 초기화
result_df = pd.DataFrame(columns=['corp_code', 'corp_name', 'sex', 'position'])

# 회사 코드 반복 처리
for i in tqdm(range(len(df_listed))):
    corp_code = df_listed['corp_code'].iloc[i]
    url = f'https://opendart.fss.or.kr/api/exctvSttus.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year=2019&reprt_code=11011'

    try:
        # DART API 호출
        response = requests.get(url)
        data = response.json()

        # 임원 정보 추출
        if 'list' in data:
            for exec_info in data['list']:
                corp_name = exec_info['corp_name']
                sex = exec_info['sexdstn']
                position = exec_info['ofcpsNm']
                result_df = result_df.append({'corp_code': corp_code, 'corp_name': corp_name, 'sex': sex, 'position': position}, ignore_index=True)
    except:
        pass

# 결과 데이터프레임 저장
result_df.to_excel('2019임원.xlsx', index=False)

In [52]:
df1 = pd.read_excel('/content/2019_임원 데이터.xlsx')

In [53]:
df1['years'] = 2019
df1.head()

,Unnamed: 0,sex_count,corp_name,rgist_exctv_at,years
0,0,1,지오엠씨,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\nNa...,2019
1,0,1,클루넷,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\n4 ...,2019
2,0,0,GRT,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\n4 ...,2019
3,0,0,보진재,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\nNa...,2019
4,0,1,로스웰,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원...,2019


In [54]:
# 컬럼명 변경 (A -> Alpha, B -> Beta)
df1 = df1.rename(columns={'sex_count': '여성 임원 수', 'corp_name' : '회사명', 'rgist_exctv_at' : '총 임원 수'})
df1. head()

,Unnamed: 0,여성 임원 수,회사명,총 임원 수,years
0,0,1,지오엠씨,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\nNa...,2019
1,0,1,클루넷,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\n4 ...,2019
2,0,0,GRT,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\n4 ...,2019
3,0,0,보진재,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\nNa...,2019
4,0,1,로스웰,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원...,2019


In [55]:
# 컬럼 삭제
df1 = df1.drop(columns='Unnamed: 0')
df1.head()

,여성 임원 수,회사명,총 임원 수,years
0,1,지오엠씨,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\nNa...,2019
1,1,클루넷,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\n4 ...,2019
2,0,GRT,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\n4 ...,2019
3,0,보진재,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원\nNa...,2019
4,1,로스웰,0 등기임원\n1 등기임원\n2 등기임원\n3 등기임원...,2019


In [56]:
df = pd.read_excel('/content/0627데이터_v007.xlsx')

In [57]:
# 'stock_code' 컬럼을 이용하여 새로운 데이터프레임 생성
df2 = df[['stock_code', 'years', '회사명']]
df2.head()

,stock_code,years,회사명
0,20,2019,동화약품
1,20,2020,동화약품
2,20,2021,동화약품
3,20,2022,동화약품
4,20,2023,동화약품


In [20]:
# 두 데이터프레임 병합 (년도와 회사명을 기준으로)
df_new = pd.merge(df1, df2, on=['years', '회사명'], how='left')

In [58]:
df_new

,여성 임원 수,회사명,years,stock_code,임원 수
42,0,우진,2019,105840.0,10
46,0,대동,2019,490.0,11
49,0,삼화콘덴서공업,2019,1820.0,11
50,0,유니온,2019,910.0,7
51,0,우진플라임,2019,49800.0,5
...,...,...,...,...,...
2447,0,한국주철관공업,2019,970.0,13
2449,0,한신공영,2019,4960.0,32
2454,0,HD한국조선해양,2019,9540.0,32
2456,0,새론오토모티브,2019,75180.0,15


In [59]:
# 'stock_code' 컬럼에서 결측치가 있는 행 제거
df_new = df_new.dropna(subset=['stock_code'], how='any')

In [61]:
df_new

,여성 임원 수,회사명,years,stock_code,임원 수
42,0,우진,2019,105840.0,10
46,0,대동,2019,490.0,11
49,0,삼화콘덴서공업,2019,1820.0,11
50,0,유니온,2019,910.0,7
51,0,우진플라임,2019,49800.0,5
...,...,...,...,...,...
2447,0,한국주철관공업,2019,970.0,13
2449,0,한신공영,2019,4960.0,32
2454,0,HD한국조선해양,2019,9540.0,32
2456,0,새론오토모티브,2019,75180.0,15


In [60]:
# '총 임원 수' 컬럼에서 '임원' 글자 개수 세기
df_new['임원 수'] = df_new['총 임원 수'].str.count('임원')

KeyError: '총 임원 수'

In [62]:
df_new.head()

,여성 임원 수,회사명,years,stock_code,임원 수
42,0,우진,2019,105840.0,10
46,0,대동,2019,490.0,11
49,0,삼화콘덴서공업,2019,1820.0,11
50,0,유니온,2019,910.0,7
51,0,우진플라임,2019,49800.0,5


In [63]:
# 컬럼명 변경 (A -> Alpha, B -> Beta)
df1 = df_new.rename(columns={'임원 수': '총 임원 수'})
df1. head()

,여성 임원 수,회사명,years,stock_code,총 임원 수
42,0,우진,2019,105840.0,10
46,0,대동,2019,490.0,11
49,0,삼화콘덴서공업,2019,1820.0,11
50,0,유니온,2019,910.0,7
51,0,우진플라임,2019,49800.0,5


In [64]:
# 'stock_code' 컬럼의 소수점 이하 숫자 제거
df1['stock_code'] = df1['stock_code'].astype(int)

In [65]:
df1.head()

,여성 임원 수,회사명,years,stock_code,총 임원 수
42,0,우진,2019,105840,10
46,0,대동,2019,490,11
49,0,삼화콘덴서공업,2019,1820,11
50,0,유니온,2019,910,7
51,0,우진플라임,2019,49800,5


In [66]:
# 여성 임원 비율 계산 및 새 컬럼 생성
df1['여성 임원 비율'] = (df1['여성 임원 수'] / df1['총 임원 수']) * 100

In [67]:
df1.loc[:,'여성 임원 수'].values

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  2,  2,  2,  0,  0,  0,  0,
        0,  0,  0,  2,  2,  0,  1,  0,  1,  0,  2,  1,  0,  0,  1,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  1,  1,  7,  4,
        0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        1,  0,  1,  1,  2,  0,  1,  0,  0,  1,  1,  2,  0,  1,  1,  1,  0,
        2,  0,  0,  0,  0,  4,  0,  2,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0,  1,  0,  1,  0,  0,  1,  0,  1,  0,  1,  0,  2, 10,
        1,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        1,  1,  0,  0,  0,  5,  0,  0,  0, 11,  0,  1,  0,  6,  2,  6,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  2,  0,  1,  0,
        2,  0,  0,  0,  1,  1,  1,  0,  0,  6,  0,  1,  0,  0,  0,  0,  3,
        1,  0,  0,  2,  0

In [122]:
df1.to_excel('2019여성임원.xlsx')

In [123]:
df = pd.read_excel('/content/0627데이터_v007.xlsx')

In [124]:
# 두 데이터프레임을 'years'와 'stock_code' 컬럼을 기준으로 병합
merged_df = pd.merge(df, df1, on=['years', 'stock_code'], how='left')

# 첫 번째 데이터프레임의 결측치를 두 번째 데이터프레임의 값으로 채우기
merged_df['총 임원 수_x'] = merged_df['총 임원 수_y']
merged_df['여성 임원 수_x'] = merged_df['여성 임원 수_y']
merged_df['여성 임원 비율_x'] = merged_df['여성 임원 비율_y']

# 결과 출력
print(merged_df)

KeyError: '여성 임원 수_y'

In [120]:
# 첫 번째 데이터프레임에서 'years' 컬럼이 2019인 행 선택
mask = (df['years'] == 2019)

# 두 번째 데이터프레임에서 'stock_code' 컬럼이 첫 번째 데이터프레임의 'stock_code'와 일치하는 행 선택
mask1 = df1['stock_code'].isin(df['stock_code'])

# 두 번째 데이터프레임에서 선택된 행의 '총 임원 수' 값을 첫 번째 데이터프레임의 결측치에 넣기
df.loc[mask, '총 임원 수'] = df1.loc[mask1, '총 임원 수']

In [121]:
df.to_excel('test.xlsx')

In [106]:
# 'years'가 '2019'인 데이터가 있는지 확인
if '2019' in df['years'].values:
    # 첫 번째 데이터프레임에서 'years'가 '2019'인 행의 인덱스 가져오기
    index_to_fill = df[df['years'] == '2019'].index

    # 인덱스가 비어있지 않은지 확인
    if not index_to_fill.empty:
        # df2에서 'years'가 '2019'에 해당하는 데이터 가져오기
        data_to_fill = df1[df1['years'] == '2019'].iloc[0]

        # df1에서 해당 인덱스에 데이터 채우기
        df.loc[index_to_fill, '총 임원 수'] = data_to_fill['총 임원 수']
        df.loc[index_to_fill, '여성 임원 수'] = data_to_fill['여성 임원 수']
        df.loc[index_to_fill, '여성 임원 비율'] = data_to_fill['여성 임원 비율']

        print("업데이트된 첫 번째 데이터프레임:")
        print(df)
    else:
        print("첫 번째 데이터프레임에서 'years'가 '2019'인 데이터가 없습니다.")
else:
    print("첫 번째 데이터프레임에서 'years'가 '2019'인 데이터가 없습니다.")

첫 번째 데이터프레임에서 'years'가 '2019'인 데이터가 없습니다.


In [94]:
result

,index,years,회사명_x,stock_code,전년도ESG,종합등급,매출액,영업이익,영업이익(발표기준),당기순이익,...,장애인고용의무인원,장애인근로자수,장애인고용률,Unnamed: 73,여성 임원 수,회사명_y,총 임원 수_y,여성 임원 비율_y,총 임원 수,여성 임원 비율
0,2019-12-01,2019,동화약품,20,NaN,B,3072.0,96,96,91,...,NaN,NaN,NaN,NaN,NaN,동화약품,21.0,9.52381,NaN,NaN
1,2020-12-01,2020,동화약품,20,B,B,2721.0,232,232,287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-01,2021,동화약품,20,B,C,2930.0,225,225,196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-01,2022,동화약품,20,C,C,3404.0,299,299,216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01,2023,동화약품,20,NaN,NaN,3611.0,188,188,282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,2023-12-01,2023,F&F,383220,NaN,NaN,19785.0,5518,5518,4250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3980,2022-12-01,2022,에코프로에이치엔,383310,NaN,B,2182.0,414,414,324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3981,2023-12-01,2023,에코프로에이치엔,383310,NaN,NaN,2289.0,418,418,335,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3982,2022-12-01,2022,LX홀딩스,383800,NaN,B+,2369.0,1589,1589,1701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 결측치 채우기
result[['여성 임원 수', '총 임원 수', '여성 임원 비율']] = result.fillna(method='ffill')

In [77]:
result

,index,years,회사명,stock_code,전년도ESG,종합등급,매출액,영업이익,영업이익(발표기준),당기순이익,...,GICS_Industry,RE100,high_carbon,sbti,상시근로자수,장애인고용의무인원,장애인근로자수,장애인고용률,Unnamed: 73,여성 임원 수
0,2019-12-01,2019,동화약품,20,NaN,B,3072.0,96.0,96.0,91.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-01,2020,동화약품,20,B,B,2721.0,232.0,232.0,287.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-01,2021,동화약품,20,B,C,2930.0,225.0,225.0,196.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-01,2022,동화약품,20,C,C,3404.0,299.0,299.0,216.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01,2023,동화약품,20,NaN,NaN,3611.0,188.0,188.0,282.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4675,NaT,2019,한국주철관공업,970,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4676,NaT,2019,한신공영,4960,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4677,NaT,2019,HD한국조선해양,9540,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4678,NaT,2019,새론오토모티브,75180,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [68]:
# 기존 데이터프레임과 새로운 데이터프레임 concat
result1 = pd.concat([df, df1], ignore_index=True)

# 중복된 행 제거 (optional)
result1 = result1.drop_duplicates(subset=['years', 'stock_code'])

In [74]:
result1

,index,years,회사명,stock_code,전년도ESG,종합등급,매출액,영업이익,영업이익(발표기준),당기순이익,...,GICS_Industry,RE100,high_carbon,sbti,상시근로자수,장애인고용의무인원,장애인근로자수,장애인고용률,Unnamed: 73,여성 임원 수
0,2019-12-01,2019,동화약품,20,NaN,B,3072.0,96.0,96.0,91.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-01,2020,동화약품,20,B,B,2721.0,232.0,232.0,287.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-01,2021,동화약품,20,B,C,2930.0,225.0,225.0,196.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-01,2022,동화약품,20,C,C,3404.0,299.0,299.0,216.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01,2023,동화약품,20,NaN,NaN,3611.0,188.0,188.0,282.0,...,Drug Manufacturers - Specialty & Generic,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,2023-12-01,2023,F&F,383220,NaN,NaN,19785.0,5518.0,5518.0,4250.0,...,Apparel Manufacturing,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3979,2022-12-01,2022,에코프로에이치엔,383310,NaN,B,2182.0,414.0,414.0,324.0,...,Not Available,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3980,2023-12-01,2023,에코프로에이치엔,383310,NaN,NaN,2289.0,418.0,418.0,335.0,...,Not Available,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3981,2022-12-01,2022,LX홀딩스,383800,NaN,B+,2369.0,1589.0,1589.0,1701.0,...,Conglomerates,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# 두 데이터프레임 병합 (년도와 회사명을 기준으로)
result = pd.merge(df, df_new, on=['years', 'stock_code'], how='left')

In [41]:
result.columns

Index(['index', 'years', '회사명_x', 'stock_code', '전년도ESG', '종합등급', '매출액',
       '영업이익', '영업이익(발표기준)', '당기순이익', '지배주주순이익', '비지배주주순이익', '자산총계', '부채총계',
       '자본총계', '지배주주지분', '비지배주주지분', '자본금', '부채비율', '유보율', '영업이익률', '지배주주순이익률',
       'ROA', 'ROE', 'EPS(원)', 'BPS(원)', 'DPS(원)', 'PER', 'PBR', '발행주식수',
       '배당수익률', '지속가능경영 보고서공시여부', '매출액_상승률(퍼센트)', '영업이익_상승률', '배당수익률_상승률',
       '영업이익률_상승률', '이사의 수', '사외이사 수', '사외이사 비율', '녹색 기업 지정', '총 임원 수',
       '여성 임원 수 ', '여성 임원 비율', '남성 평균 연봉', '여성 평균 연봉', '남성 정규직 수', '여성 정규직 수',
       '남성 직원 수', '여성 직원 수', '직원 수', '여성 직원 비율', '남성 직원 비율', '남성 정규직 비율',
       '여성 정규직 비율', '외국인 보유수량', '외국인 지분율', '시가총액', '상장주식수', '용수 사용량(Ton)',
       '에너지 총량(TJ)', '온실가스 배출총량(TonCo2eq)', '용수 사용량 등급', '에너지 총량 등급',
       '온실가스 배출총량 등급', 'GICS_Sector', 'GICS_Industry', 'RE100', 'high_carbon',
       'sbti', '상시근로자수', '장애인고용의무인원', '장애인근로자수', '장애인고용률', 'Unnamed: 73',
       '여성 임원 수', '회사명_y', '임원 수'],
      dtype='object')

In [42]:
result.head()

,index,years,회사명_x,stock_code,전년도ESG,종합등급,매출액,영업이익,영업이익(발표기준),당기순이익,...,high_carbon,sbti,상시근로자수,장애인고용의무인원,장애인근로자수,장애인고용률,Unnamed: 73,여성 임원 수,회사명_y,임원 수
0,2019-12-01,2019,동화약품,20,NaN,B,3072.0,96,96,91,...,0,0,NaN,NaN,NaN,NaN,NaN,2.0,동화약품,21.0
1,2020-12-01,2020,동화약품,20,B,B,2721.0,232,232,287,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-12-01,2021,동화약품,20,B,C,2930.0,225,225,196,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-01,2022,동화약품,20,C,C,3404.0,299,299,216,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-12-01,2023,동화약품,20,NaN,NaN,3611.0,188,188,282,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
